# Week2

### week2_오다건_세션과제
### 1. 아래에 주어진 주석을 기반으로 하여 코딩을 해주세요.

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pylab as plt
import random

In [2]:
#파라미터 설정

learning_rate = 0.1
training_epochs = 15
batch_size = 100

In [4]:
#train 과 test set으로 나누어 MNIST 데이터 불러오기

mnist_train = dsets.MNIST(root='MNIST_data/',
                         train=True,
                         transform = transforms.ToTensor(),
                         download =True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                        train=False,
                        transform = transforms.ToTensor(),
                        download=True)

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw


Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw




Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw
Processing...
Done!


/Users/distiller/project/conda/conda-bld/pytorch_1591914925853/work/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [7]:
#dataset loader에 train 과 test 할당하기.
#batch size, shuffle(순서를 무작위로),drop_last(크기에 맞지 않는 데이터 drop할지 안할지)

train_loader = torch.utils.data.DataLoader(dataset=mnist_train, 
                                          batch_size = batch_size,
                                          shuffle=True,
                                          drop_last = True)

test_loader = torch.utils.data.DataLoader(dataset=mnist_test, 
                                          batch_size = batch_size,
                                          shuffle=False,
                                          drop_last = True)


In [10]:
# Layer 쌓기 (조건: 3개의 Layer 사용, DropOut 사용 (p=0.3), ReLU 함수 사용, Batch normalization하기) 
# 각 Layer의 Hidden node 수 : 1st Layer (784,100), 2nd Layer(100,100),3rd Layer(100,10)

linear1 = torch.nn.Linear(784,100, bias=True)
linear2 = torch.nn.Linear(100,100, bias=True)
linear3 = torch.nn.Linear(100,10, bias=True)

relu = torch.nn.ReLU()
bn1 = torch.nn.BatchNorm1d(100) #number of size
bn2 = torch.nn. BatchNorm1d(100)


dropout = torch.nn.Dropout(p=0.3)


In [11]:
#xavier initialization 이용해서 각 layer weight 초기화
#xavier initialization => 시그모이드의 양 극단이 0으로 수렴해서 역전파의 과정에서
#기울기 소실의 문제가 발생할 수  있음.

torch.nn.init.xavier_uniform(linear1.weight)
torch.nn.init.xavier_uniform(linear2.weight)
torch.nn.init.xavier_uniform(linear3.weight)

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  """
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  import sys


Parameter containing:
tensor([[-1.6294e-01,  2.1647e-01, -5.8171e-02,  1.8560e-02,  1.5128e-01,
          1.9623e-01,  2.0730e-01,  2.1863e-01, -2.4716e-02,  1.1893e-01,
         -1.1394e-02, -2.0730e-01,  2.0492e-01, -1.5781e-01, -1.7326e-01,
         -1.7959e-01, -1.1447e-01, -1.4627e-02, -1.3109e-01,  2.9577e-02,
         -1.4077e-01, -4.0010e-02,  1.9457e-01, -1.7065e-01, -1.6669e-02,
         -2.0035e-01, -1.8170e-02,  2.2263e-02, -7.1494e-02,  1.5544e-02,
          1.5608e-01, -1.2987e-01,  8.9370e-02, -5.8783e-02,  1.7699e-01,
         -1.6194e-01,  1.0401e-01, -5.8772e-02, -1.2016e-01, -9.3409e-02,
         -1.5245e-01,  1.9454e-01, -5.7787e-02, -1.1542e-01, -2.2468e-01,
         -1.8898e-01, -3.0636e-02, -3.6729e-02,  1.4153e-01, -1.2008e-01,
         -1.6520e-01,  1.1044e-01, -9.8183e-02,  1.7162e-01, -4.7478e-02,
          1.7293e-02, -1.0792e-01,  1.1774e-01, -1.6785e-01,  1.4957e-01,
         -1.5212e-01,  1.4670e-01,  1.3533e-01, -1.7918e-01, -2.3033e-01,
         -1.3298

In [24]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# CUDA 장치 객체(device object)로

In [25]:
#torch.nn.Sequential 을 이용하여  model 정의하기
#쌓는 순서에 주의.
#linear-batch normalization-relu-dropout 순으로 반복.

model = torch.nn.Sequential(linear1,  bn1, relu, dropout, 
                           linear2, bn2, relu,  dropout,
                            linear3).to(device)

In [27]:
#Loss function 정의 하기 (Cross entropy)
#optimzer - Adam optimizer

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [31]:
#cost 계산 위한 변수 설정
train_total_batch  = len(train_loader)

#Training epoch(cost 값 초기 설정0으로, model의 train 설정 꼭 할것.

model.train()

for epoch in range(training_epochs):
    avg_cost = 0
    
#train dataset을 불러오고(X,Y 불러오기), 
#back propagation과 optimizer를 사용하여 loss를 최적화하는 코드

    for X,Y in train_loader:
        X= X.view(-1,28*28).to(device)
        Y= Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost +=cost/ train_total_batch
        
    print('Epoch:','%04d' % (epoch+1),'cost=', '{:.9f}'.format(avg_cost))
    
print('Learning finished.')

    

Epoch: 0001 cost= 0.497178823
Epoch: 0002 cost= 0.375539780
Epoch: 0003 cost= 0.336459905
Epoch: 0004 cost= 0.313253313
Epoch: 0005 cost= 0.305459440
Epoch: 0006 cost= 0.294139832
Epoch: 0007 cost= 0.281584591
Epoch: 0008 cost= 0.277942628
Epoch: 0009 cost= 0.266553968
Epoch: 0010 cost= 0.263346672
Epoch: 0011 cost= 0.259189874
Epoch: 0012 cost= 0.244044974
Epoch: 0013 cost= 0.254103214
Epoch: 0014 cost= 0.240941405
Epoch: 0015 cost= 0.238476470
Learning finished.


In [40]:
#test data로 모델의 정확도를 검증하는 코드 (model의 evaluation mode 설정 꼭 할 것) 
#X_test 불러올 때 view를 사용하여 차원 변환할 것
#Y_test를 불러올때 labels사용 #accuracy의 초기 값 설정(0으로) 꼭 할 것


with torch.no_grad():
    model.eval() #예측할때는 사용하지 않으니까.
    
    #test dataset의 데이터 형태를 batch*784로 바꾸어줌
    X_test = mnist_test.test_data.view(-1,28*28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    
    #각 batch별로 가장 높은 가능성의 숫자 클래스를 뽑아주기.
    correct_prediction = torch.argmax(prediction,1)==Y_test
    accuracy = correct_prediction.float().mean() #맞는 개수의 평균을 내면
    print('Accuracy:', accuracy.item()) #정확도가 나온다. 
    
    #get one and predict 랜덤으로 뽑아서 하나만 예측해보자
    
    r = random.randint(0,len(mnist_test)-1)
    X_single_data = mnist_test.test_data[r:r+1].view(-1,28*28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r+1].to(device)
    
    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())
    
    
    
    

Accuracy: 0.9269999861717224
Label:  7
Prediction:  7


### 2.Layer의 hidden node 수를 증가 혹은 감소시켰을 때, train set에서의 cost와 test set에서의 Accuracy가 기존 결과와 비교하였을 때 어떻게 달라졌는지 비교


### 784-> 200-> 100

In [41]:
linear4= torch.nn.Linear(784,200, bias=True)
linear5= torch.nn.Linear(200,100, bias=True)
linear6 = torch.nn.Linear(100,10, bias=True)

relu = torch.nn.ReLU()
bn3 = torch.nn.BatchNorm1d(200)
bn4 =  torch.nn.BatchNorm1d(100)
dropout = torch.nn.Dropout(p=0.3)

torch.nn.init.xavier_uniform_(linear4.weight)
torch.nn.init.xavier_uniform_(linear5.weight)
torch.nn.init.xavier_uniform_(linear6.weight)



Parameter containing:
tensor([[-0.0879, -0.0918,  0.0112,  0.0019, -0.2209, -0.1594, -0.0656,  0.1329,
         -0.0562, -0.2196, -0.2279,  0.1385,  0.0089, -0.1217,  0.0519, -0.1211,
          0.1869,  0.2175,  0.2267,  0.1387, -0.1145,  0.0657,  0.1789,  0.2026,
          0.0292, -0.0987, -0.0872,  0.0947, -0.1448, -0.2189, -0.1077, -0.1147,
         -0.1978,  0.1763,  0.0064,  0.0233,  0.1345, -0.1491,  0.1576, -0.1130,
          0.0499,  0.1779, -0.1573, -0.1153,  0.0318, -0.1944,  0.2128, -0.1594,
          0.2301,  0.1047, -0.1202, -0.0605, -0.2318, -0.2067,  0.1726, -0.1689,
         -0.1225, -0.1825,  0.0040,  0.1257,  0.1591, -0.0585,  0.1599,  0.2235,
          0.0474, -0.1111, -0.1522,  0.0158,  0.2093,  0.1773,  0.1766,  0.2012,
         -0.1817, -0.0970, -0.1383,  0.2292, -0.2226, -0.0837,  0.1164,  0.0331,
         -0.0129, -0.1305,  0.0035,  0.1715,  0.1862,  0.1772, -0.1008, -0.0050,
          0.1602,  0.0857,  0.1926,  0.0030,  0.0464,  0.2316, -0.1492,  0.1802,
      

In [42]:
model_1= torch.nn.Sequential(linear4,bn3,relu,dropout,linear5,bn4, relu,dropout,linear6).to(device)


In [43]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model_1.parameters(),lr=learning_rate)
train_total_batch = len(train_loader)

In [44]:
model_1.train()

for epoch in range(training_epochs):
    avg_cost = 0

    for X,Y in train_loader:
        X = X.view(-1, 28*28).to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model_1(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        avg_cost += cost/ train_total_batch
    
    print('Epoch:','%04d' % (epoch+1),'cost=', '{:.9f}'.format(avg_cost))

print('Learning finished.')

    

Epoch: 0001 cost= 0.453175068
Epoch: 0002 cost= 0.323464662
Epoch: 0003 cost= 0.303170592
Epoch: 0004 cost= 0.272519827
Epoch: 0005 cost= 0.251503080
Epoch: 0006 cost= 0.247146651
Epoch: 0007 cost= 0.232310325
Epoch: 0008 cost= 0.227555573
Epoch: 0009 cost= 0.218427002
Epoch: 0010 cost= 0.226032585
Epoch: 0011 cost= 0.211749524
Epoch: 0012 cost= 0.198414698
Epoch: 0013 cost= 0.210980415
Epoch: 0014 cost= 0.195357725
Epoch: 0015 cost= 0.192274213
Learning finished.


In [52]:
with torch.no_grad():
    model_1.eval() #예측할때는 사용하지 않으니까.
    
    #test dataset의 데이터 형태를 batch*784로 바꾸어줌
    X_test = mnist_test.test_data.view(-1,28*28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model_1(X_test)
    
    #각 batch별로 가장 높은 가능성의 숫자 클래스를 뽑아주기.
    correct_prediction = torch.argmax(prediction,1)==Y_test
    accuracy = correct_prediction.float().mean() #맞는 개수의 평균을 내면
    print('Accuracy:', accuracy.item()) #정확도가 나온다. 
    
    #get one and predict 랜덤으로 뽑아서 하나만 예측해보자
    
    r = random.randint(0,len(mnist_test)-1)
    X_single_data = mnist_test.test_data[r:r+1].view(-1,28*28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r+1].to(device)
    
    print('Label: ', Y_single_data.item())
    single_prediction = model_1(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.9247000217437744
Label:  3
Prediction:  3


## 784->150->50

In [59]:
linear7= torch.nn.Linear(784,150, bias=True)
linear8= torch.nn.Linear(150,50, bias=True)
linear9 = torch.nn.Linear(50,10, bias=True)

relu = torch.nn.ReLU()
bn5 = torch.nn.BatchNorm1d(150)
bn6 =  torch.nn.BatchNorm1d(50)

dropout = torch.nn.Dropout(p=0.3)

torch.nn.init.xavier_uniform_(linear7.weight)
torch.nn.init.xavier_uniform_(linear8.weight)
torch.nn.init.xavier_uniform_(linear9.weight)



Parameter containing:
tensor([[ 0.2821,  0.2988, -0.2087, -0.2392,  0.0100,  0.2381,  0.2699,  0.2631,
          0.2873, -0.1684, -0.1720, -0.3120, -0.2315, -0.2136, -0.2963,  0.0095,
         -0.2428,  0.2021,  0.0036, -0.1789, -0.0574,  0.2057,  0.2784, -0.2236,
         -0.1016, -0.0589, -0.2068,  0.1926,  0.1011, -0.0814, -0.1486, -0.1977,
          0.1376,  0.3112,  0.1931,  0.2115, -0.2705, -0.1597, -0.0578,  0.1773,
          0.2604, -0.2885, -0.0566, -0.0798,  0.1289, -0.2123,  0.0270,  0.2695,
         -0.0945,  0.1202],
        [ 0.0057,  0.0930, -0.2098, -0.0623, -0.3081,  0.2761,  0.0706,  0.1768,
         -0.0204, -0.1397,  0.2735, -0.1565, -0.2934,  0.2608, -0.0427,  0.0917,
          0.1451, -0.1644,  0.0518,  0.1837,  0.0575, -0.2443,  0.0947, -0.0735,
         -0.0586, -0.1019, -0.1296,  0.0569, -0.1884, -0.1935,  0.1656,  0.0816,
          0.0009, -0.2533,  0.3006, -0.2782, -0.0441, -0.1077, -0.0186,  0.2070,
         -0.0375, -0.2339, -0.3158,  0.2258,  0.2048, -0.27

In [61]:
model_2= torch.nn.Sequential(linear7,bn5,relu,dropout,linear8,bn6, relu,dropout,linear9).to(device)

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model_2.parameters(),lr=learning_rate)
train_total_batch = len(train_loader)

In [62]:
model_2.train()

for epoch in range(training_epochs):
    avg_cost = 0

    for X,Y in train_loader:
        X = X.view(-1, 28*28).to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model_2(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        avg_cost += cost/ train_total_batch
    
    print('Epoch:','%04d' % (epoch+1),'cost=', '{:.9f}'.format(avg_cost))

print('Learning finished.')

Epoch: 0001 cost= 0.482527375
Epoch: 0002 cost= 0.337119251
Epoch: 0003 cost= 0.299585283
Epoch: 0004 cost= 0.291280389
Epoch: 0005 cost= 0.278631896
Epoch: 0006 cost= 0.263238013
Epoch: 0007 cost= 0.241790026
Epoch: 0008 cost= 0.237411425
Epoch: 0009 cost= 0.230461031
Epoch: 0010 cost= 0.233719453
Epoch: 0011 cost= 0.222534403
Epoch: 0012 cost= 0.221500278
Epoch: 0013 cost= 0.215621382
Epoch: 0014 cost= 0.210736290
Epoch: 0015 cost= 0.196814656
Learning finished.


In [63]:
with torch.no_grad():
    model_2.eval() #예측할때는 사용하지 않으니까.
    
    #test dataset의 데이터 형태를 batch*784로 바꾸어줌
    X_test = mnist_test.test_data.view(-1,28*28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    
    #각 batch별로 가장 높은 가능성의 숫자 클래스를 뽑아주기.
    correct_prediction = torch.argmax(prediction,1)==Y_test
    accuracy = correct_prediction.float().mean() #맞는 개수의 평균을 내면
    print('Accuracy:', accuracy.item()) #정확도가 나온다. 
    
    #get one and predict 랜덤으로 뽑아서 하나만 예측해보자
    
    r = random.randint(0,len(mnist_test)-1)
    X_single_data = mnist_test.test_data[r:r+1].view(-1,28*28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r+1].to(device)
    
    print('Label: ', Y_single_data.item())
    single_prediction = model_2(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.9269999861717224
Label:  5
Prediction:  5
